In [101]:
import unidic_lite, fugashi
from transformers import pipeline

In [102]:
# sentiment_analyzer = pipeline("sentiment-analysis", model="daigo/bert-base-japanese-sentiment",token=)

sentiment_analyzer = pipeline("text-classification", model="bennexx/cl-tohoku-bert-base-japanese-v3-jlpt-classifier")

Device set to use mps:0


In [103]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="kit-nlp/bert-base-japanese-sentiment-irony")

Device set to use mps:0


In [104]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "bennexx/cl-tohoku-bert-base-japanese-v3-jlpt-classifier"

# トークナイザの読み込み
tokenizer = AutoTokenizer.from_pretrained(model_name)

# モデルの読み込み
model = AutoModelForSequenceClassification.from_pretrained(model_name)

name = input()
import pandas as pd

alpha_df = pd.read_csv('../TalkData/alpha_'+name+'.csv')
beta_df = pd.read_csv('../TalkData/beta_'+name+'.csv')

output_alpha = pd.read_csv('../AnalyzedData/analyzed_alpha_'+name+'.csv')
output_beta = pd.read_csv('../AnalyzedData/analyzed_beta_'+name+'.csv')
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "bennexx/cl-tohoku-bert-base-japanese-v3-jlpt-classifier"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def predict_jlpt_level(text: str) -> int:
    """
    文章を入力すると、モデルの推定するJLPTレベルIDを返す。
    例: 0: N5, 1: N4, 2: N3, 3: N2, 4: N1
    """
    # トークナイズ
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    # デバイスに載せる
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_id = torch.argmax(logits, dim=-1).item()
    
    return predicted_id

# 新たに 'JLPTLevel' カラムを作成し、各行の "Detail" に対する推定結果を埋める
output_alpha["JLPTLevel"] = output_alpha["Detail"].apply(predict_jlpt_level)
output_beta["JLPTLevel"] = output_beta["Detail"].apply(predict_jlpt_level)

output_alpha.to_csv("../AnalyzedData/analyzed_alpha_"+name+".csv", index=False)
output_beta.to_csv("../AnalyzedData/analyzed_beta_"+name+".csv", index=False)